# OpenMC Tallies

In this tutorial, we will learn how to:

  - Understand application of filters and scores to create tallies
  - Apply tallies to an OpenMC simulation
  - Extract information from OpenMC statepoint files
  - Understand tally units and normalization
  - Plot tally results
  - Control the number of batches to reach desired statistical conditions 

In [ ]:
import openmc
import matplotlib.pyplot as plt
import numpy as np

In this section, we'll be looking at how to extract custom information from an OpenMC simulation in what is known as a "tally." A tally accumulates statistical information during the simulation about particles when they eneter regions of phase space specified on the tally. The limits of these regions are set by "filters" applied to the tally. Scores and nuclides can also be applied to tallies to indicate what type of information is kept about the particle (e.g. reaction types, flux, heat, etc.).

Any tally in OpenMC can be described with the following form:

$$ 
 X = \underbrace{\int d\mathbf{r} \int d\mathbf{\Omega} \int
    dE}_{\text{filters}} \underbrace{f(\mathbf{r}, \mathbf{\Omega},
    E)}_{\text{scores}} \underbrace{\psi (\mathbf{r}, \mathbf{\Omega}, E)}_{\text{angular flux}}
$$

where filters set the limits of the integrals and the scoring function is convolved with particle information (e.g. reaction type, current material, etc.). For example, if you wanted to calculate the fission reaction rate caused by fast neutrons in cell 3, your tally becomes

$$ 
 X = \int_\text{cell 3} d\mathbf{r} \int_{4\pi} d\mathbf{\Omega} \int_{1 MeV}^{20 MeV}
    dE \ \ \Sigma_f(\mathbf{r}, \mathbf{\Omega},
    E) \psi (\mathbf{r}, \mathbf{\Omega}, E)
$$

<div class="alert alert-block alert-info">
A full list of scores and their meanings can be found <a href=https://docs.openmc.org/en/stable/usersguide/tallies.html#scores >here</a>.
</div>

## Pincell Model

First we'll need a model to examine. OpenMC has a few basic models that we can use to look at tally setup. The function below generates a 2-D PWR pincell model with reflective boundary conditions on the X-Y planes. This function provides an `openmc.Model` object, which ties together materials, geometry, tallies, and settings in a single Python object with a full problem description.

In [ ]:
model = openmc.examples.pwr_pin_cell()

To get a better idea of what this model looks like, we'll start by generating a plot and examining the materials used.

In [ ]:
model.geometry.root_universe.plot()

In [ ]:
model.geometry.get_all_materials()

If we look at the tallies object on our pincell model, we'll see there aren't currently any custom tallies applied.

In [ ]:
model.tallies

In this exercise we'll be adding tallies to perform a few different tasks:


  **1. Determine the average energy produced per fission** \
  **2. Plot the flux spectrum of the pincell** \
  **3. Plot reaction types based on material**
  
To do this we'll use a variety of different filters applied to different tallies.

## An aside on units

<div class="alert alert-block alert-info">

Geometry units specified in the model build are always in __cm__. Volumes computed by OpenMC will be in __cm<sup>3</sup>__. Tally values for energy will always be reported in __eV__.

Tally values are always reported __per source particle__. A "source particle" is a physical particle which exists in the "real world" - *not* a simulated particle that you control with `settings.particles`. Tallies in OpenMC should be normalized by the source strength $S$, in $\frac{\text{src}}{\text{s}}$. For example, 

Reacion rate: $r \left\lbrack\frac{\text{reactions}}{\text{src}}\right\rbrack * S \left\lbrack\frac{\text{src}}{\text{s}}\right\rbrack \rightarrow \left\lbrack\frac{\text{reactions}}{\text{s}}\right\rbrack$

Flux tally: $t \left\lbrack\frac{\text{particle-cm}}{\text{src}}\right\rbrack * \frac{{\text{S}}}{\text{V}} \left\lbrack\frac{\text{src}}{\text{s}} \frac{1}{\text{cm}^3}\right\rbrack \rightarrow \left\lbrack\frac{\text{particle}}{\text{cm}^2\text{-s}}\right\rbrack$
</b>

For example,

- If you have a fixed source photon transport problem with a 1 Ci source of photons, then your source particle rate in the "real world" is $S=3.7\times10^{10}$ src/s.
- If you have a fixed source photon transport problem with a 1 Ci source, but only 85% of the decays produce a photon, then your source particle rate in the "real world" is $S=0.85*3.7\times10^{10}$ src/s.
- If you have an eigenvalue problem, then you need to impose some reaction rate from which you can determine $S$. For example, if you know your reactor produces $p$ Wth, then you would determine the source rate as

Heating tally: $r \left\lbrack\frac{\text{eV}}{\text{src}}\right\rbrack * \textcolor{red}{S} \left\lbrack\frac{\text{src}}{\text{s}}\right\rbrack = p \left\lbrack\frac{J}{\text{s}}\right\rbrack * \frac{1}{1.602\times10^{-19}} \left\lbrack\frac{eV}{\text{J}}\right\rbrack $

- For eigenvalue problems, knowing the total power is the most common way to normalize, but is not strictly necessary - *any* reaction rate will do! You'll just construct a different equation to determine $S$. Suppose you had a radiation detector in the core from which you know the neutron-induced fission rate $d$. If you added this fission rate as a tally, then you can find $S$ by

Fission reaction rate in detector: $r \left\lbrack\frac{\text{reaction}}{\text{src}}\right\rbrack * \textcolor{red}{S} \left\lbrack\frac{\text{src}}{\text{s}}\right\rbrack = d \left\lbrack\frac{\text{reaction}}{\text{s}}\right\rbrack  $

## Energy released per fission

To compute the energy released per fission, we will use two different scores - the `kappa-fission` score, which tallies the recoverable energy release from fission [eV/src], and the `fission` score, which tallies the fission rate [fission reactions/src]. The energy released per fission, averaged over all fission events, is simply the `kappa-fission` score divided by the `fission` score. We start with this tally, because if your quantity of interest is a ratio of two other tallies, we may not have to do any normalization with a source rate.

Because we want this information talllied throughout the model, a "global" tally, no filters need to be applied.

In [ ]:
fission_tally = openmc.Tally()
fission_tally.scores = ['fission', 'kappa-fission']
print(fission_tally)

In [ ]:
model.tallies = openmc.Tallies([fission_tally])


After adjusting the default settings for number of particles and batches on the model we'll run it and examine the data.

In [ ]:
model.settings.batches = 50
model.settings.inactive = 10
model.settings.particles = 10000
statepoint = model.run()

If we list our current directory, we see that several new files have been created as a result of this run: `summary.h5`, `tallies.out`, and `statepoint.50.h5`. The summary file contains information about the simulation's setup (geometry, materials, meshes, etc.) in an HDF5 format. The `tallies.out` file contains a text output of all user-specified tallies for the simulation.

In [ ]:
!cat tallies.out

This can be useful to quickly look at simple tally results, but isn't a great format to post-process simulation data. For that we'll look to the statepoint file. The statepoint file contains information about simulation results including tally specifications and data. The location of this statepoint file was provided to us by the `model.run()` command.

In [ ]:
print(statepoint)

To extract information from the statepoint file we'll create an `openmc.StatePoint` object. The `statepoint.get_tally` function will search for tallies by scores, filters, nuclides, ids, and return the closest match. Exact matches can be specified as well.

In [ ]:
with openmc.StatePoint(statepoint) as sp:
    tally_by_scores = sp.get_tally(scores=['fission'])
    tally_by_id = sp.get_tally(id=fission_tally.id)

If we print the tally objects returned, we see that they indeed match the tally specification we generated above. The `tally_by_scores` represents the tally object, so even though we only searched for the fission score, the quantity we extracted includes the tally as a whole entity (not just the one score).

In [ ]:
print(tally_by_scores)

In [ ]:
print(tally_by_id)

In [ ]:
# filter, nuclide, score
tally_by_id.shape

In [ ]:
tally_by_scores.shape

<div class="alert alert-block alert-info">
<b>A quick aside on how statepoint objects interact with summary files:</b>


The `openmc.statepoint` object will read information from the `summary.h5` file if one is present, keeping that file open in the Python interpreter. The open `summary.h5` file can interfere with the initialization of subsequent OpenMC simulations. It is recommended that information be extracted from statepoints within a [context manager](https://book.pythontips.com/en/latest/context_managers.html) as we do here. Alternatively, making sure to call the `openmc.StatePoint.close` method will work also. For more details please look to the [relevant section in the user's guide](https://docs.openmc.org/en/stable/usersguide/troubleshoot.html#runtimeerror-failed-to-open-hdf5-file-with-mode-w-summary-h5).   
</div>

To compute the energy released per fission event, we can simply take the tallied energy released per fission and divide it by the fission rate. `squeeze()` is a python function that will eliminate axes of length 1 (for us, these are the slices pertaining to material filters or nuclides).

In [ ]:
fission_rate = tally_by_id.get_values(scores=['fission']).squeeze()
kappa_fission_rate = tally_by_id.get_values(scores=['kappa-fission']).squeeze()

ev_per_fission = kappa_fission_rate / fission_rate
mev_per_fission = ev_per_fission * 1e-6

print('MeV per fission: ', mev_per_fission)

For a water reactor with U235 as the only fissioning isotope this is about what we would expect: ~193 MeV! 

### Uncertainties

When dealing with tallies, we must remember that every output of a Monte Carlo simulation is uncertain -- it is associated with a mean and a standard deviation. Whenever you present results from a Monte Carlo simulation, you should *always* present the mean value *and* its standard deviation. To obtain the standard deviation associated with our MeV/fission estimation, we can use the Python [uncertainties](https://pythonhosted.org/uncertainties/) module, while also using the `value='std_dev'` option when fetching the tally values to get the standard deviations.

In [ ]:
fission_rate_std_dev = tally_by_id.get_values(scores=['fission'], value='std_dev').flatten()[0]
kappa_fission_rate_std_dev = tally_by_id.get_values(scores=['kappa-fission'],value='std_dev').flatten()[0]

from uncertainties import ufloat
f = ufloat(fission_rate, fission_rate_std_dev)
kf = ufloat(kappa_fission_rate, kappa_fission_rate_std_dev)
ratio = kf / f * 1e-6

print('MeV per fission: {:.6f}'.format(ratio))

It is common to communicate these uncertainties in terms of a "relative error," or the standard deviation divided by the mean.

In [ ]:
rel_err = ratio.std_dev / ratio.nominal_value
print('Relative error (%): {:.2f}'.format(100 * rel_err))

## Plot the neutron flux spectrum


Plotting a neutron flux spectrum is a very useful way to understand the physical processes happening to neutrons - the energy at which they exist is determined by scattering reactions (to lower energies), capture reactions (removing them from the population), as well as their birth distribution (such as from fission or fusion). It is often an engineer's objective to control the energies at which neutrons are predominantly at in their system in order to encourage certain reactions.

To plot the neutron flux spectrum, we'll be applying a tally with an energy filter and a score. OpenMC's data module contains different group structures. For this problem we'll use the CASMO-70 group structure. An energy filter can easily be created from a pre-defined group structure in OpenMC as follows:

In [ ]:
print(openmc.mgxs.GROUP_STRUCTURES.keys())

In [ ]:
energy_filter = openmc.EnergyFilter.from_group_structure('CASMO-70')
len(energy_filter.bins)

In [ ]:
spectrum_tally = openmc.Tally()
spectrum_tally.filters = [energy_filter]
spectrum_tally.scores = ['flux']
print(spectrum_tally)

Now we'll apply this tally and re-run the problem. We can leave the other tally we added earlier by appending our additional tally.

In [ ]:
model.tallies += [spectrum_tally]
statepoint = model.run(output=False)

As before, we can fetch our tally of interest by finding the closest match based on the ID, scores, and/or filters.

In [ ]:
with openmc.StatePoint(statepoint) as sp:
    tally_by_id = sp.get_tally(id=spectrum_tally.id)
    tally_by_scores = sp.get_tally(scores=['flux'])
    tally_by_filters = sp.get_tally(filters=spectrum_tally.filters)

In [ ]:
spectrum = tally_by_id.mean
print(spectrum.shape)

spectrum = spectrum.squeeze()
print(spectrum.shape)

Now to plot the spectrum, we will plot the neutron flux per unit lethargy (a common way to visualize neutron flux). We will plot the flux spectrum with a point at the lower energy bin value of each bin (you could alternatively plot in the midpoint or the right point defining each bin).

$\text{lethargy bin width}\equiv\ln\frac{E_i}{E_{i-1}}$

In [ ]:
unit_lethargy = energy_filter.lethargy_bin_width

In [ ]:
print(energy_filter.bins)

In [ ]:
plt.step(np.unique(energy_filter.bins)[:-1], spectrum / unit_lethargy)
plt.xscale('log')
plt.xlabel('Energy (eV)')
plt.ylabel('Flux per unit lethargy (particle-cm/src/lethargy)')
plt.show()

## Normalizing Tallies

Note that the units of flux in the above plot are in $\frac{\text{particle-cm}}{\text{src}}$ per unit lethargy. As is the case with many values tallied by Monte Carlo codes, the value of the flux does not account for volume and is in terms of the number of source particles emitted in the "real world". To generate this same plot in terms of absolute flux units ($\frac{\text{particle}}{\text{cm}^{2}-\text{s}}$) we'll need to normalize this tally by:

  - the volume of the region the tally covers
  - the number of source particle emitted per second, $S$

In this case, the volume of the region is the volume of the entire pincell, because we did not add any spatial filters. Because we're working with a 2-D model, we'll get units that give us the flux per unit length of the pincell in the axial direction. For simplicity, we'll assume that our pincell is 1 cm in height to make life easier.

In order to obtain the volumes of the cells, we can

- Code in the volume of the region analytically; this is trivial for our pincell ($p^2H$, where $p$ is the pitch), but may not be possible for more general shapes.
- If it is relevant to your problem, you can use a bounding box which covers the relevant domain.
- Perform a stochastic volume calculation.

In [ ]:
lower_left, upper_right = model.geometry.bounding_box
print(lower_left, upper_right)

In [ ]:
lower_left[-1] = 0.0
upper_right[-1] = 1.0

volume = np.prod(upper_right - lower_left)
print(volume)

Determining the number of source particles per second requires us to impose knowledge of some reaction rate. Let's choose to specify the total power produced in the pincell. The quantity $S$ is what we seek in order to multiply against our flux tally. So, we need to have a heating tally over the same domain as the flux tally to give us $r$. Luckily, we already have this tally added to our simulation from the earlier portion.

Heating tally: $r \left\lbrack\frac{\text{eV}}{\text{src}}\right\rbrack * \textcolor{red}{S} \left\lbrack\frac{\text{src}}{\text{s}}\right\rbrack = p \left\lbrack\frac{J}{\text{s}}\right\rbrack * \frac{1}{1.602\times10^{-19}} \left\lbrack\frac{eV}{\text{J}}\right\rbrack $

In [ ]:
with openmc.StatePoint(statepoint) as sp:
    source_tally = sp.get_tally(scores=['kappa-fission'])
    
idx = source_tally.get_score_index('kappa-fission')
source = source_tally.get_values().squeeze()[idx]

In [ ]:
source = source_tally.get_slice(scores=['kappa-fission']).get_values().squeeze()

In [ ]:
power = 200 # Watts
neutron_source = power / 1.602e-19 * (1 / source)
print(f'Neutron source: {neutron_source:.2e} n/s')

We can now use this information to normalize our flux values and reproduce our plot in more standard units. The shape of the plot is identical to what we obtained earlier, as all that we've done is scale the y-axis into more conventional units for flux. We can also plot $\pm3\sigma$ on our plot, though the error bars are visibly small due to the large range in values shown on the y-axis.

In [ ]:
normalized_spectrum = spectrum * neutron_source / volume

In [ ]:
spectrum_std_dev = tally_by_id.std_dev.squeeze() * neutron_source / volume

In [ ]:
e = np.unique(energy_filter.bins)
plt.step(e[:-1], normalized_spectrum / unit_lethargy, where='mid')

plt.errorbar(e[:-1], normalized_spectrum / unit_lethargy, yerr=3*spectrum_std_dev, capsize=2, fmt='None')
plt.xscale('log')
plt.xlabel('Energy (eV)')
plt.ylabel('Flux per unit lethargy (1/cm$^2$/s/lethargy)')
plt.show()

## Reaction Types by Material

Looking at the different reaction types by material will require a material filter and the set of reaction types we want to score. For this example, we'll be scoring absorption, scattering and fission in each material. To start, we'll create a material filter. We'll add all our materials to this filter so that we obtain reaction rates separately for each.

In [ ]:
model.materials

In [ ]:
material_filter = openmc.MaterialFilter(model.materials)

In [ ]:
material_tally = openmc.Tally()
material_tally.filters = [material_filter]
material_tally.scores = ['absorption', 'scatter', 'fission']

In [ ]:
model.tallies += [material_tally]

In [ ]:
statepoint = model.run()

Now we'll gather information from the statepoint file about each score we applied to the tally. With multiple scores and materials, we'll use a Pandas data frame to view the results in a more coherent manner.

In [ ]:
with openmc.StatePoint(statepoint) as sp:
    tally = sp.get_tally(id=material_tally.id)
    df = tally.get_pandas_dataframe()

Each score has three values -- one for each material in the model.

In [ ]:
df

First, we'll add a new column to the data frame with normalized results.

In [ ]:
df['normalized-mean (rxn/s)'] = neutron_source * df['mean']
df

We'll add a new entry in the dataframe for our material names to make plotting easier.

In [ ]:
materials = model.geometry.get_all_materials()
type(materials)

In [ ]:
# set names based on matching material IDs
for mat_id, material in materials.items():
    df.loc[df['material'] == mat_id, 'mat_name'] = material.name
df

In [ ]:
fission_df = df[df['score'] == 'fission']
fission_df

In [ ]:
fission_df.plot('mat_name', 'normalized-mean (rxn/s)', kind='bar', ylabel='fissions / s')

In [ ]:
scatter_df = df[df['score'] == 'scatter']
scatter_df.plot('mat_name', 'mean', kind='bar', ylabel='scatters / s')

In [ ]:
absorption_df = df[df['score'] == 'absorption']
absorption_df.plot('mat_name', 'mean', kind='bar', ylabel='absorptions / s')

## Tally Triggers

When running OpenMC, you usually want to run enough computational resources (particles, batches) to adequately reduce the statistical error in your predictions. If the tally realizations are independent of one another, then the standard deviation decreases as

$\sigma\propto\frac{1}{\sqrt{N}}$

You can use this approximate relationship to sketch out how many batches are required to reach a given statistical threshold - but this can be tedious and requires running OpenMC at least twice. A better way to proceed is to use a `Trigger`, which will continue running batches in OpenMC until a desired condition is met on the standard deviation, variance, and/or relative error.

- $\sigma<\sigma_{tol}$
- $\sigma^2<v_{tol}$
- $\frac{\sigma}{\mu}<r_{tol}$

Triggers can be applied to (i) any tally you create or (ii) the automatically-applied $k_{eff}$ tally which OpenMC creates internally. The approach is slightly different for each.
Let's start with a trigger on $k$. For $k$, we add a trigger using the `model.settings.keff_trigger` parameter. OpenMC will re-evaluate the projected number of batches required, assuming the central limit theorem holds, every `model.settings.trigger_batch_interval` batches. Since OpenMC could run forever, you should set the `model.settings.trigger_max_batches` to be the maximum number of batches to run (terminating at that point even if the trigger is not met); the minimum number of batches which will be run is the `model.settings.batches`.

In [ ]:
model.settings.keff_trigger = {'type' : 'std_dev', 'threshold' : 0.00100}
model.settings.trigger_active = True
model.settings.trigger_batch_interval = 10
model.settings.trigger_max_batches = 1000

statepoint = model.run()

To add a trigger now for a generic tally, we create an `openmc.Trigger` object and apply it to a tally. When multiple triggers are used, OpenMC prints out the trigger which is the furthest from convergence.

In [ ]:
rel_err_trig = openmc.Trigger(trigger_type='rel_err', threshold=1e-3)
fission_tally.triggers = [rel_err_trig]

In [ ]:
statepoint = model.run()

In [ ]:
with openmc.StatePoint(statepoint) as sp:
    means = sp.get_tally(scores=['kappa-fission']).get_values(value='mean').squeeze()
    std_devs = sp.get_tally(scores=['kappa-fission']).get_values(value='std_dev').squeeze()

rel_errs = std_devs / means
print(rel_errs)